In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df_air=pd.read_csv('qualite-de-lair-mesuree-dans-la-station-franklin-d-roosevelt.csv',delimiter=';')
df_air

,date/heure,NO,NO2,PM10,CO2,TEMP,HUMI
0,2021-10-18T02:00:00+02:00,9,44,34,516,"20,3","40,9"
1,2021-10-18T01:00:00+02:00,7,46,35,512,"20,5","40,6"
2,2021-10-18T00:00:00+02:00,6,44,42,522,"20,7","40,5"
3,2021-10-17T23:00:00+02:00,7,40,48,532,"20,2","40,8"
4,2021-10-17T22:00:00+02:00,5,37,50,551,"19,9","39,6"
...,...,...,...,...,...,...,...
76865,2013-01-01T06:00:00+01:00,27,57,15,464,15,"63,3"
76866,2013-01-01T05:00:00+01:00,27,56,22,481,"14,9","63,7"
76867,2013-01-01T04:00:00+01:00,22,52,45,490,"14,9","63,3"
76868,2013-01-01T03:00:00+01:00,43,61,18,536,"14,9",65


In [3]:
df_air.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76870 entries, 0 to 76869
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date/heure  76870 non-null  object
 1   NO          71691 non-null  object
 2   NO2         72231 non-null  object
 3   PM10        73947 non-null  object
 4   CO2         72085 non-null  object
 5   TEMP        74968 non-null  object
 6   HUMI        74961 non-null  object
dtypes: object(7)
memory usage: 4.1+ MB


**1 : conversion de la colonne date/heure en format datetime**

In [4]:
df_air['date/heure'].str.extract('[^+]*\+(.*)').value_counts()

02:00    45694
01:00    31176
dtype: int64

Le format UTC+1 correspond à l'heure d'hiver et le format UTC+2 correspond à l'heure d'été.
On veut raisonner en heure "ressentie" donc on fait sauter la notion d'UTC.

In [5]:
df_air['date/heure']=df_air['date/heure'].str.extract('([^+]*)')
df_air['date/heure']=pd.to_datetime(df_air['date/heure'])
df_air

,date/heure,NO,NO2,PM10,CO2,TEMP,HUMI
0,2021-10-18 02:00:00,9,44,34,516,"20,3","40,9"
1,2021-10-18 01:00:00,7,46,35,512,"20,5","40,6"
2,2021-10-18 00:00:00,6,44,42,522,"20,7","40,5"
3,2021-10-17 23:00:00,7,40,48,532,"20,2","40,8"
4,2021-10-17 22:00:00,5,37,50,551,"19,9","39,6"
...,...,...,...,...,...,...,...
76865,2013-01-01 06:00:00,27,57,15,464,15,"63,3"
76866,2013-01-01 05:00:00,27,56,22,481,"14,9","63,7"
76867,2013-01-01 04:00:00,22,52,45,490,"14,9","63,3"
76868,2013-01-01 03:00:00,43,61,18,536,"14,9",65


In [6]:
df_air.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76870 entries, 0 to 76869
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date/heure  76870 non-null  datetime64[ns]
 1   NO          71691 non-null  object        
 2   NO2         72231 non-null  object        
 3   PM10        73947 non-null  object        
 4   CO2         72085 non-null  object        
 5   TEMP        74968 non-null  object        
 6   HUMI        74961 non-null  object        
dtypes: datetime64[ns](1), object(6)
memory usage: 4.1+ MB


In [7]:
df_air=df_air.sort_values('date/heure')
df_air=df_air.reset_index(drop=True)

**2. Gestion des valeurs non numériques pour obtenir un df contenant uniquement de float et des Nan en plus de date/time**

Il y a des NaN (qui correspondent aux valeurs initiales vides) et des ND issus de la BD initiale : on convertit tout en Nan

In [8]:
df_air.isna().sum()

date/heure       0
NO            5179
NO2           4639
PM10          2923
CO2           4785
TEMP          1902
HUMI          1909
dtype: int64

In [9]:
(df_air=='ND').sum()

date/heure       0
NO            3513
NO2           2688
PM10          1175
CO2           2777
TEMP            30
HUMI            30
dtype: int64

In [10]:
df_air=df_air.replace('ND',np.nan)

In [11]:
df_air.isna().sum()

date/heure       0
NO            8692
NO2           7327
PM10          4098
CO2           7562
TEMP          1932
HUMI          1939
dtype: int64

In [12]:
print(f'Pourcentage de valeurs NaN (dont ND converties en Nan) : {round(df_air.isna().sum().sum()/df_air.size*100,1)}%')

Pourcentage de valeurs NaN (dont ND converties en Nan) : 5.9%


In [13]:
print(f'Pourcentage de lignes contenant au moins un NaN (dont ND converties en NaN) : {round(df_air.isnull().any(axis=1).sum()/df_air.shape[0]*100,1)}%')

Pourcentage de lignes contenant au moins un NaN (dont ND converties en NaN) : 19.6%


In [14]:
df_air.loc[df_air['NO'].isna()].sort_values('date/heure').loc[728:760]

,date/heure,NO,NO2,PM10,CO2,TEMP,HUMI
730,2013-01-31 12:00:00,NaN,NaN,63,NaN,"17,7","49,4"
731,2013-01-31 13:00:00,NaN,NaN,98,556,"18,6","48,1"
732,2013-01-31 14:00:00,NaN,NaN,76,526,19,"47,6"
733,2013-01-31 15:00:00,NaN,NaN,69,515,19,"50,9"
734,2013-01-31 16:00:00,NaN,NaN,26,502,"18,5","51,5"
735,2013-01-31 17:00:00,NaN,NaN,27,510,"18,2","53,7"
736,2013-01-31 18:00:00,NaN,NaN,26,537,18,"51,6"
737,2013-01-31 19:00:00,NaN,NaN,40,559,"18,3","48,3"
738,2013-01-31 20:00:00,NaN,NaN,50,553,"18,3","46,9"
739,2013-01-31 21:00:00,NaN,NaN,45,513,"18,6","45,5"


On voit ci-dessus que les valeurs manquantes ne sont pas forcément éparpillées et peuvent être nombreuses à la suite

On s'intérese maintenant aux valeurs dépassant les seuils des capteurs, qui sont sous la forme <5, >1000, <2 : on va considérer que leur valeur est égale au seuil du capteur

In [15]:
df_air['PM10'].value_counts().sort_index()

10       789
100      100
101      121
102      104
103      120
        ... 
979        1
98       109
99       110
<5       202
>1000      1
Name: PM10, Length: 417, dtype: int64

In [16]:
df_air[['NO','PM10']]=df_air[['NO','PM10']].applymap(lambda x : re.search('(\d+)',x).group(1)  if pd.notnull(x) else np.nan)

In [17]:
df_air['PM10'].value_counts().sort_index()

10      789
100     100
1000      1
101     121
102     104
       ... 
96      126
97      133
979       1
98      109
99      110
Name: PM10, Length: 416, dtype: int64

On termine par remplacer les , par des .

In [18]:
df_air[['TEMP','HUMI']].head()

,TEMP,HUMI
0,"15,1","65,6"
1,"14,9",65
2,"14,9","63,3"
3,"14,9","63,7"
4,15,"63,3"


In [19]:
df_air[['TEMP','HUMI']]=df_air[['TEMP','HUMI']].applymap(lambda x : x.replace(',','.') if pd.notnull(x) else np.nan)

On peut enfin tout convertir en float

In [20]:
df_air[df_air.columns[1:]]=df_air[df_air.columns[1:]].astype(float)

In [21]:
df_air.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76870 entries, 0 to 76869
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date/heure  76870 non-null  datetime64[ns]
 1   NO          68178 non-null  float64       
 2   NO2         69543 non-null  float64       
 3   PM10        72772 non-null  float64       
 4   CO2         69308 non-null  float64       
 5   TEMP        74938 non-null  float64       
 6   HUMI        74931 non-null  float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 4.1 MB


**3. Verification des doublons sur les dates/heure et suppression pour passage en index des date/heure**

In [22]:
df_air[df_air['date/heure'].duplicated(keep=False)]

,date/heure,NO,NO2,PM10,CO2,TEMP,HUMI
7175,2013-10-27 02:00:00,57.0,79.0,27.0,445.0,21.8,53.3
7176,2013-10-27 02:00:00,16.0,48.0,21.0,407.0,21.3,54.4
15910,2014-10-26 02:00:00,114.0,62.0,33.0,523.0,18.9,62.7
15911,2014-10-26 02:00:00,120.0,63.0,29.0,503.0,18.5,64.1
24646,2015-10-25 02:00:00,51.0,58.0,40.0,444.0,18.1,64.2
24647,2015-10-25 02:00:00,41.0,51.0,32.0,415.0,18.4,64.5
33550,2016-10-30 02:00:00,46.0,59.0,27.0,496.0,19.8,51.2
33551,2016-10-30 02:00:00,41.0,55.0,25.0,468.0,19.7,50.7
42262,2017-10-29 02:00:00,24.0,54.0,22.0,444.0,20.5,49.9
42263,2017-10-29 02:00:00,20.0,49.0,18.0,416.0,20.6,51.2


A chaque changement d'heure, on a un doublon sur le 2h du matin (9 au tota). On fait la moyenne de ces deux valeurs identiques car on ne peut pas l'intercaler, toutes les heures étant déjà présentes - Aucun Nan sur ces données donc pas de soucis pour moyenner.

In [23]:
df_air=df_air.groupby('date/heure').mean()
df_air

,NO,NO2,PM10,CO2,TEMP,HUMI
date/heure,,,,,,
2013-01-01 02:00:00,23.0,41.0,26.0,612.0,15.1,65.6
2013-01-01 03:00:00,43.0,61.0,18.0,536.0,14.9,65.0
2013-01-01 04:00:00,22.0,52.0,45.0,490.0,14.9,63.3
2013-01-01 05:00:00,27.0,56.0,22.0,481.0,14.9,63.7
2013-01-01 06:00:00,27.0,57.0,15.0,464.0,15.0,63.3
...,...,...,...,...,...,...
2021-10-17 22:00:00,5.0,37.0,50.0,551.0,19.9,39.6
2021-10-17 23:00:00,7.0,40.0,48.0,532.0,20.2,40.8
2021-10-18 00:00:00,6.0,44.0,42.0,522.0,20.7,40.5


Vérification que les doublons ont bien sauté :

In [24]:
df_air.loc['2014-10-26 00:00':'2014-10-26 04:00']

,NO,NO2,PM10,CO2,TEMP,HUMI
date/heure,,,,,,
2014-10-26 00:00:00,149.0,79.0,44.0,554.0,19.1,62.5
2014-10-26 01:00:00,108.0,64.0,36.0,544.0,19.1,62.3
2014-10-26 02:00:00,117.0,62.5,31.0,513.0,18.7,63.4
2014-10-26 03:00:00,114.0,60.0,22.0,494.0,18.6,63.2
2014-10-26 04:00:00,145.0,57.0,24.0,528.0,18.7,63.0


In [25]:
df_air.loc['2016-12-31':'2017-01-01']

,NO,NO2,PM10,CO2,TEMP,HUMI
date/heure,,,,,,
2016-12-31 00:00:00,203.0,85.0,41.0,566.0,9.4,44.4
2016-12-31 01:00:00,214.0,90.0,37.0,539.0,9.2,44.3
2017-01-01 02:00:00,12.0,39.0,139.0,457.0,7.6,43.2
2017-01-01 03:00:00,67.0,54.0,73.0,498.0,8.1,42.6
2017-01-01 04:00:00,80.0,58.0,47.0,493.0,7.7,41.8
2017-01-01 05:00:00,53.0,50.0,51.0,473.0,7.6,41.6
2017-01-01 06:00:00,43.0,45.0,35.0,469.0,7.6,41.5
2017-01-01 07:00:00,50.0,48.0,26.0,459.0,7.4,40.9
2017-01-01 08:00:00,31.0,40.0,25.0,446.0,6.9,42.2


In [26]:
df_air.isna().sum()

NO      8691
NO2     7326
PM10    4096
CO2     7561
TEMP    1931
HUMI    1938
dtype: int64

**4. Vérification des trous sur la série temporelle**

Ci-dessous, un exemple d'heure manquante comme on pouvait s'y attendre lors du changement d'heure du printemps 2020 - Ainsi qu'une autre valeur plus étonnante à 4h

In [27]:
df_air.loc['2020-03-29 00:00':'2020-03-29 05:00']

,NO,NO2,PM10,CO2,TEMP,HUMI
date/heure,,,,,,
2020-03-29 00:00:00,NaN,NaN,24.0,NaN,19.9,32.9
2020-03-29 01:00:00,NaN,NaN,15.0,NaN,19.1,33.6
2020-03-29 03:00:00,NaN,NaN,11.0,NaN,19.0,32.1
2020-03-29 05:00:00,NaN,NaN,11.0,NaN,18.6,31.8


In [28]:
# index contenant toutes les heures de la toute première à la toute dernière date_heure présente dans le dataframe
full_index = pd.date_range(start=df_air.index[0], end=df_air.index[-1], freq='H')
full_index

DatetimeIndex(['2013-01-01 02:00:00', '2013-01-01 03:00:00',
               '2013-01-01 04:00:00', '2013-01-01 05:00:00',
               '2013-01-01 06:00:00', '2013-01-01 07:00:00',
               '2013-01-01 08:00:00', '2013-01-01 09:00:00',
               '2013-01-01 10:00:00', '2013-01-01 11:00:00',
               ...
               '2021-10-17 17:00:00', '2021-10-17 18:00:00',
               '2021-10-17 19:00:00', '2021-10-17 20:00:00',
               '2021-10-17 21:00:00', '2021-10-17 22:00:00',
               '2021-10-17 23:00:00', '2021-10-18 00:00:00',
               '2021-10-18 01:00:00', '2021-10-18 02:00:00'],
              dtype='datetime64[ns]', length=77089, freq='H')

In [29]:
len(set(full_index)-set(df_air.index))

228

In [30]:
len(full_index)-len(df_air.index)

228

Liste des 228 valeurs absentes de notre jeux de données ci-dessous

In [31]:
set(full_index)-set(df_air.index)

{Timestamp('2013-03-31 02:00:00', freq='H'),
 Timestamp('2014-03-30 02:00:00', freq='H'),
 Timestamp('2014-07-01 02:00:00', freq='H'),
 Timestamp('2015-03-29 02:00:00', freq='H'),
 Timestamp('2016-03-27 02:00:00', freq='H'),
 Timestamp('2016-12-31 02:00:00', freq='H'),
 Timestamp('2016-12-31 03:00:00', freq='H'),
 Timestamp('2016-12-31 04:00:00', freq='H'),
 Timestamp('2016-12-31 05:00:00', freq='H'),
 Timestamp('2016-12-31 06:00:00', freq='H'),
 Timestamp('2016-12-31 07:00:00', freq='H'),
 Timestamp('2016-12-31 08:00:00', freq='H'),
 Timestamp('2016-12-31 09:00:00', freq='H'),
 Timestamp('2016-12-31 10:00:00', freq='H'),
 Timestamp('2016-12-31 11:00:00', freq='H'),
 Timestamp('2016-12-31 12:00:00', freq='H'),
 Timestamp('2016-12-31 13:00:00', freq='H'),
 Timestamp('2016-12-31 14:00:00', freq='H'),
 Timestamp('2016-12-31 15:00:00', freq='H'),
 Timestamp('2016-12-31 16:00:00', freq='H'),
 Timestamp('2016-12-31 17:00:00', freq='H'),
 Timestamp('2016-12-31 18:00:00', freq='H'),
 Timestamp

Deux catégories de valeurs manquantes :

1. Changements d'heure et quelques très rares valeurs seules manquantes --> on va réindexer et interpoler

2. Périodes de durée non négligeable --> on laisse des trous dans le dataframe
- 2016-12-31 02:00 à 2017-01-01 01:00 --> 24 valeurs/H
- 2018-12-31 02:00 à 2019-01-01 00:00 --> 23 valeurs/H
- 2021-03-22 02:00 à 2021-03-29 02:00 --> 7 jours et 1H = 169 valeurs

In [32]:
new_index=pd.date_range(df_air.index[0],'2016-12-31 01:00',freq='H')
new_index=new_index.append(pd.date_range('2017-01-01 02:00','2018-12-31 01:00',freq='H'))
new_index=new_index.append(pd.date_range('2019-01-01 01:00','2021-03-22 01:00',freq='H'))
new_index=new_index.append(pd.date_range('2021-03-29 03:00',df_air.index[-1],freq='H'))
new_index
# Etape inutile car ces lignes auraient sauté avec le interpolate limit suivi du dropna

DatetimeIndex(['2013-01-01 02:00:00', '2013-01-01 03:00:00',
               '2013-01-01 04:00:00', '2013-01-01 05:00:00',
               '2013-01-01 06:00:00', '2013-01-01 07:00:00',
               '2013-01-01 08:00:00', '2013-01-01 09:00:00',
               '2013-01-01 10:00:00', '2013-01-01 11:00:00',
               ...
               '2021-10-17 17:00:00', '2021-10-17 18:00:00',
               '2021-10-17 19:00:00', '2021-10-17 20:00:00',
               '2021-10-17 21:00:00', '2021-10-17 22:00:00',
               '2021-10-17 23:00:00', '2021-10-18 00:00:00',
               '2021-10-18 01:00:00', '2021-10-18 02:00:00'],
              dtype='datetime64[ns]', length=76873, freq=None)

In [33]:
df_air=df_air.reindex(new_index)
df_air=df_air.sort_index()

Des lignes complètement vides ont été ajoutées au niveau des index manquants

In [34]:
df_air.loc['2020-03-29 00:00':'2020-03-29 05:00']

,NO,NO2,PM10,CO2,TEMP,HUMI
2020-03-29 00:00:00,NaN,NaN,24.0,NaN,19.9,32.9
2020-03-29 01:00:00,NaN,NaN,15.0,NaN,19.1,33.6
2020-03-29 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-29 03:00:00,NaN,NaN,11.0,NaN,19.0,32.1
2020-03-29 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-29 05:00:00,NaN,NaN,11.0,NaN,18.6,31.8


**5. On élimine les NaN en interpolant (uniquement si il ne manque pas trop de valeurs consécutives d'un coup)**

In [35]:
print(f'Pourcentage de valeurs NaN (dont ND converties en Nan) : {round(df_air.isna().sum().sum()/df_air.size*100,1)}%')

Pourcentage de valeurs NaN (dont ND converties en Nan) : 6.9%


In [36]:
print(f'Pourcentage de lignes contenant au moins un NaN (dont ND converties en NaN) : {round(df_air.isnull().any(axis=1).sum()/df_air.shape[0]*100,1)}%')

Pourcentage de lignes contenant au moins un NaN (dont ND converties en NaN) : 19.6%


In [37]:
df_air.isnull().sum()

NO      8703
NO2     7338
PM10    4108
CO2     7573
TEMP    1943
HUMI    1950
dtype: int64

Sur des cas comme celui ci-dessous, il semble logique d'interpoler pour conserver un maximum de lignes

In [38]:
df_air.loc['2014-02-28 00:00':'2014-02-28 06:00']

,NO,NO2,PM10,CO2,TEMP,HUMI
2014-02-28 00:00:00,36.0,76.0,16.0,497.0,13.7,53.3
2014-02-28 01:00:00,28.0,70.0,12.0,480.0,13.5,51.8
2014-02-28 02:00:00,19.0,59.0,13.0,451.0,13.1,52.4
2014-02-28 03:00:00,6.0,39.0,NaN,444.0,13.2,52.1
2014-02-28 04:00:00,6.0,39.0,NaN,447.0,13.1,52.7
2014-02-28 05:00:00,5.0,34.0,5.0,446.0,12.5,55.9
2014-02-28 06:00:00,10.0,42.0,10.0,454.0,12.9,55.7


In [39]:
df_air=df_air.sort_index()
df_air=df_air.interpolate(limit=2) # On interpole jusqu'à un certain nombre de NaN consécutifs. Au dela, on ne fait rien.

In [40]:
df_air.loc['2014-02-28 00:00':'2014-02-28 06:00']

,NO,NO2,PM10,CO2,TEMP,HUMI
2014-02-28 00:00:00,36.0,76.0,16.000000,497.0,13.7,53.3
2014-02-28 01:00:00,28.0,70.0,12.000000,480.0,13.5,51.8
2014-02-28 02:00:00,19.0,59.0,13.000000,451.0,13.1,52.4
2014-02-28 03:00:00,6.0,39.0,10.333333,444.0,13.2,52.1
2014-02-28 04:00:00,6.0,39.0,7.666667,447.0,13.1,52.7
2014-02-28 05:00:00,5.0,34.0,5.000000,446.0,12.5,55.9
2014-02-28 06:00:00,10.0,42.0,10.000000,454.0,12.9,55.7


In [41]:
df_air.isnull().sum()

NO      7503
NO2     6785
PM10    2604
CO2     6524
TEMP    1821
HUMI    1828
dtype: int64

In [42]:
print(f'Pourcentage de valeurs NaN (dont ND converties en Nan) : {round(df_air.isna().sum().sum()/df_air.size*100,1)}%')

Pourcentage de valeurs NaN (dont ND converties en Nan) : 5.9%


In [43]:
print(f'Pourcentage de lignes contenant au moins un NaN (dont ND converties en NaN) : {round(df_air.isnull().any(axis=1).sum()/df_air.shape[0]*100,1)}%')

Pourcentage de lignes contenant au moins un NaN (dont ND converties en NaN) : 16.2%


In [44]:
df_air.shape[0]-df_air.isnull().any(axis=1).sum()

64390

**6. On supprime les lignes qui contiennent toujours des NaN et on repasse les colonnes NO à CO2 en int (c'est la précision du capteur)**

In [45]:
df_air=df_air.dropna()
df_air

,NO,NO2,PM10,CO2,TEMP,HUMI
2013-01-01 02:00:00,23.0,41.0,26.0,612.0,15.1,65.6
2013-01-01 03:00:00,43.0,61.0,18.0,536.0,14.9,65.0
2013-01-01 04:00:00,22.0,52.0,45.0,490.0,14.9,63.3
2013-01-01 05:00:00,27.0,56.0,22.0,481.0,14.9,63.7
2013-01-01 06:00:00,27.0,57.0,15.0,464.0,15.0,63.3
...,...,...,...,...,...,...
2021-10-17 22:00:00,5.0,37.0,50.0,551.0,19.9,39.6
2021-10-17 23:00:00,7.0,40.0,48.0,532.0,20.2,40.8
2021-10-18 00:00:00,6.0,44.0,42.0,522.0,20.7,40.5
2021-10-18 01:00:00,7.0,46.0,35.0,512.0,20.5,40.6


On repasse les colonnes de NO à CO2 en int, posible maintenant qu'on a plus de Nan (va tronquer les valeurs interpolées et moyennées)

In [46]:
df_air[['NO','NO2','PM10','CO2']]=df_air[['NO','NO2','PM10','CO2']].astype(int)

In [47]:
df_air.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 64390 entries, 2013-01-01 02:00:00 to 2021-10-18 02:00:00
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   NO      64390 non-null  int64  
 1   NO2     64390 non-null  int64  
 2   PM10    64390 non-null  int64  
 3   CO2     64390 non-null  int64  
 4   TEMP    64390 non-null  float64
 5   HUMI    64390 non-null  float64
dtypes: float64(2), int64(4)
memory usage: 3.4 MB


In [55]:
df_air

,NO,NO2,PM10,CO2,TEMP,HUMI
2013-01-01 02:00:00,23,41,26,612,15.1,65.6
2013-01-01 03:00:00,43,61,18,536,14.9,65.0
2013-01-01 04:00:00,22,52,45,490,14.9,63.3
2013-01-01 05:00:00,27,56,22,481,14.9,63.7
2013-01-01 06:00:00,27,57,15,464,15.0,63.3
...,...,...,...,...,...,...
2021-10-17 22:00:00,5,37,50,551,19.9,39.6
2021-10-17 23:00:00,7,40,48,532,20.2,40.8
2021-10-18 00:00:00,6,44,42,522,20.7,40.5
2021-10-18 01:00:00,7,46,35,512,20.5,40.6


La différence entre le nombre de lignes ici et avec la fonction vient du fait qu'ici j'ai manuellement coupé deux "trous" sur les date/heure, ce que je n'ai pas fait dans la fonction qui extrapole donc les deux premières valeurs de chaque trou (soit 4 valeurs de différence)